In [3]:
import numpy as np
import pandas as pd
import torch 
import tensorboard

Generate dataset

- Consider the exponential function f(x)=exp(-x/c), with x=0,1,2,….,L-1 integers
- Generate many training datasets {f(0,f(1),f(2),…f(L-1)} for different values of c
- Now we solve the inverse problem: input c and see if you get the exponential.

In [65]:
#Returns a dataframe of calculated exponential functions for given C-parameters and L-angular quantum numbers
class GenerateData():
    def __init__(self, l):
        self.l = l
        self.x = np.arange(l)
        self.c = np.arange(start=0.0001, stop=1, step = 0.0001)
    
    #Evaluate function for given c-values
    def Func(self, input, arg):
        #func= np.zeros(shape = (self.l,))
        #for i in range(0, self.l):
        #    func[i] = np.e**(-self.x[i]/arg)
        return np.e**(-input/arg)
    
    #Create dataset: 100,000 funcs?
    def Populate(self):
        func_df = pd.DataFrame(index = self.c.tolist())
        for i in self.x:
            col = []
            for j in self.c:
                col.append(GenerateData(self.l).Func(i,j))
            func_df[i] = col
        return func_df
    
    #Set param value for integer L-value (angular momentum quantum number)
    def Data(self):
        return GenerateData(self.l).Populate()

Dataframe for Exponential Function Values: Rows = C-values, Columns = Integer X-values

In [66]:
GenerateData(4).Data()

,0,1,2,3
0.0001,1.0,0.000000,0.000000,0.000000
0.0002,1.0,0.000000,0.000000,0.000000
0.0003,1.0,0.000000,0.000000,0.000000
0.0004,1.0,0.000000,0.000000,0.000000
0.0005,1.0,0.000000,0.000000,0.000000
...,...,...,...,...
0.9995,1.0,0.367695,0.135200,0.049712
0.9996,1.0,0.367732,0.135227,0.049727
0.9997,1.0,0.367769,0.135254,0.049742
0.9998,1.0,0.367806,0.135281,0.049757


Initialize neural net

In [61]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [62]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=3, bias=True)
  )
)


In [19]:
X = torch.rand(100, 1, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])
